**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")
#nu nog naar Oud in deze mapm gaan 
data_folder = os.path.join(data_folder, "OUD")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Kleine_data.xlsx")
df = pd.read_excel(file_path)

In [4]:
# Drop unnecessary columns
if "TXT_file_name" in df.columns:
    df = df.drop(columns=["TXT_file_name"])

# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Remove patterns like 'a)', 'b)', etc.
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    return text

# df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text) 

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

#amount of rows 
print(f"Number of rows after filtering: {len(df)}")

Number of rows after filtering: 921


C:\Users\corne\AppData\Local\Temp\ipykernel_19300\4055204460.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


In [5]:
print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())

All theme_ids: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33)]
num_labels: 34


In [6]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [7]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()
median_count = theme_counts.median()

# Define strategy: only oversample underrepresented classes
sampling_strategy = {
    theme: int(median_count)
    for theme in theme_counts.index
    if theme_counts[theme] < median_count
}

# Apply RandomOverSampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(train_df[["clean_text"]], train_df["theme_id"])

# Extract oversampled train lists
train_texts_resampled = X_resampled["clean_text"].tolist()
train_labels_resampled = y_resampled.tolist()

from collections import Counter
print("Class distribution after oversampling:", Counter(train_labels_resampled))


Class distribution after oversampling: Counter({6: 199, 15: 62, 3: 53, 2: 49, 4: 38, 16: 33, 0: 30, 13: 30, 12: 26, 21: 22, 10: 20, 11: 20, 7: 18, 14: 17, 1: 16, 28: 11, 26: 10, 19: 8, 30: 8, 5: 8, 32: 8, 31: 8, 20: 8, 27: 8, 24: 8, 8: 8, 25: 8, 9: 8, 18: 8, 23: 8, 22: 8, 29: 8, 17: 8, 33: 8})


In [8]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "GroNLP/bert-base-dutch-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts_resampled, train_labels_resampled, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [9]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 12%|█▎        | 99/792 [00:20<02:11,  5.26it/s]

{'loss': 3.0625, 'grad_norm': 10.260591506958008, 'learning_rate': 1.7500000000000002e-05, 'epoch': 1.0}


                                                
 12%|█▎        | 99/792 [00:21<02:11,  5.26it/s]

{'eval_loss': 2.8146800994873047, 'eval_accuracy': 0.2702702702702703, 'eval_precision': 0.8027757487216947, 'eval_recall': 0.2702702702702703, 'eval_f1': 0.11500862564692352, 'eval_runtime': 1.093, 'eval_samples_per_second': 169.252, 'eval_steps_per_second': 21.957, 'epoch': 1.0}


 25%|██▌       | 198/792 [00:44<02:00,  4.92it/s]

...

{'eval_loss': 2.4522008895874023, 'eval_accuracy': 0.3945945945945946, 'eval_precision': 0.5426872102734172, 'eval_recall': 0.3945945945945946, 'eval_f1': 0.3196785550621868, 'eval_runtime': 1.57, 'eval_samples_per_second': 117.836, 'eval_steps_per_second': 15.287, 'epoch': 8.0}


100%|██████████| 792/792 [03:29<00:00,  4.93it/s]

{'train_runtime': 209.4418, 'train_samples_per_second': 30.175, 'train_steps_per_second': 3.781, 'train_loss': 1.9635659420129024, 'epoch': 8.0}


100%|██████████| 792/792 [03:29<00:00,  3.78it/s]


TrainOutput(global_step=792, training_loss=1.9635659420129024, metrics={'train_runtime': 209.4418, 'train_samples_per_second': 30.175, 'train_steps_per_second': 3.781, 'total_flos': 1663339635179520.0, 'train_loss': 1.9635659420129024, 'epoch': 8.0})

SAVE MODEL TO KUL DRIVE

In [10]:
from datetime import datetime

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
save_path = f"C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP/Run_{run_id}"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


# Add a description of the model and training details
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
geen context meegegeven in zowel train als test
"""

# Save the description to a text file
description_file = f"{save_path}/model_description.txt"
with open(description_file, "w", encoding="utf-8") as f:
    f.write(description)


In [11]:
import os
from datetime import datetime
import numpy as np
import pandas as pd

# === Create timestamped save path in OneDrive ===
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
base_path = "C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP"
save_path = os.path.join(base_path, f"Run_{run_id}")
os.makedirs(save_path, exist_ok=True)

# === Save model using Trainer ===
trainer.save_model(save_path)

# === Get predictions ===
preds_output = trainer.predict(test_dataset)
logits = preds_output.predictions
predictions = np.argmax(logits, axis=1)

# === Save predictions to CSV ===
output_df = pd.DataFrame({
    "text": test_texts,  # make sure test_texts is defined
    "true_label": test_labels,  # make sure test_labels is defined
    "predicted_label": predictions,
    "logits": logits.tolist()
})

csv_path = os.path.join(save_path, "test_predictions.csv")
output_df.to_csv(csv_path, index=False)

# === Optional: Save a description file ===
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
[Add your description here, e.g., which layers were frozen, data splits, notes...]
"""

desc_path = os.path.join(save_path, "model_description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(description)

print(f"Everything saved in: {save_path}")


100%|██████████| 24/24 [00:01<00:00, 23.17it/s]

Everything saved in: C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP\Run_2025-04-10_12-18
